# Scraping reviews from Reclame Aqui website

## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
#import matplotlib.pyplot as plt
#%matplotlib inline

## Create driver

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

## Define Functions

In [3]:
def create_url(company, page):
    return f'https://www.reclameaqui.com.br/empresa/{company}/lista-reclamacoes/?pagina={page}'

In [4]:
def get_soup(driver,url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    return soup

In [5]:
def get_complaints(soup):
    complaints = soup.find_all(class_="link-complain-id-complains")
    return complaints

In [6]:
def get_titles(complaints):
    title_list = []
    for i in range(len(complaints)):
        title_list.append(complaints[i].text.strip())
    return title_list

In [7]:
def get_links(complaints):
    link_list = []
    start_url = 'https://www.reclameaqui.com.br'
    for i in range(len(complaints)):
        link = (start_url + complaints[i].get('href'))
        link_list.append(link)
    return link_list

In [8]:
def get_content(soup):
    try:
        content = soup.find_all('p', class_="ng-binding")[9].text.strip()
        return content
    except:
        return 'NA'

In [9]:
def get_titles_and_links(company, num_pages, driver):
    titles = []
    links = []
    for i in range(1, num_pages+1):
        page_url = create_url(company, i)
        page_soup = get_soup(driver, page_url)
        page_complaints = get_complaints(page_soup)
        page_titles = get_titles(page_complaints)
        page_links = get_links(page_complaints)
        
        titles += page_titles
        links += page_links
        
    return titles, links

In [10]:
def get_content_list(driver, links):
    content_list = []
    for i in range(len(links)):
        url = links[i]
        soup = get_soup(driver,url)
        print(f'Retrieved {i+1} of {len(links)}')
        content = get_content(soup)
        content_list.append(content)
    return content_list

## Test Functions

In [11]:
titles, links = get_titles_and_links('upnid',2,driver)

In [12]:
titles

['[Editado pelo Reclame Aqui]',
 'Site Shopmakee não aparece mais',
 'Comprei um kit de alongamento e ainda não enviaram',
 'Compra sem confirmação por e-mail',
 'Compra sem retorno',
 'MODELADOR ALUVI AIRWRAP 5 EM 1 × 1',
 'Produto não entregue',
 'Comprei e não recebi',
 'Não  recebi o produto',
 'compra TV global']

In [13]:
links

['https://www.reclameaqui.com.br/upnid/titulo_PelzEXcDwMywOdRF/',
 'https://www.reclameaqui.com.br/upnid/site-shopmakee-nao-aparece-mais_ZCQCbWAfaWGCRHEI/',
 'https://www.reclameaqui.com.br/upnid/comprei-um-kit-de-alongamento-e-ainda-nao-enviaram_rNOlHUMMhnjoJZ55/',
 'https://www.reclameaqui.com.br/upnid/compra-sem-confirmacao-por-e-mail_02uYTGiVZrw6m2j8/',
 'https://www.reclameaqui.com.br/upnid/compra-sem-retorno_8S7y0uPl-xtVztJc/',
 'https://www.reclameaqui.com.br/upnid/modelador-aluvi-airwrap-5-em-1-1_nBOsoPXOHWfNHHll/',
 'https://www.reclameaqui.com.br/upnid/produto-nao-entregue_ROOh8Dv36k2GZMy2/',
 'https://www.reclameaqui.com.br/upnid/comprei-e-nao-recebi_Yu3dwgC7JUEsG4T9/',
 'https://www.reclameaqui.com.br/upnid/nao-recebi-o-produto_08dtax5MJuIRP75O/',
 'https://www.reclameaqui.com.br/upnid/compra-tv-global_6fe6i3TBUozMqFED/']

In [14]:
contents = get_content_list(driver,links)

Retrieved 1 of 10
Retrieved 2 of 10
Retrieved 3 of 10
Retrieved 4 of 10
Retrieved 5 of 10
Retrieved 6 of 10
Retrieved 7 of 10
Retrieved 8 of 10
Retrieved 9 of 10
Retrieved 10 of 10


In [15]:
contents

['Fiz uma compra em uma loja chamada Montreall dia 16/12/20, questinei o prazo e me falaram 8 semanas. Bom ai vi que era [Editado pelo Reclame Aqui]. O boleto foi emitido pela upnid e nao recebi extorno nem meu produto. Quero uma posiçao porque assim como eu vi varios comprando no facebook.',
 'Comprei um modelador de cabelo no shopmakee no dia 28/02 com pagamento pela Upnid e não consigo mais encontrar o site e nem contato com a empresa para saber do produto ou pedir a devolução do pagamento.',
 'Comprei e paguei no cartão de crédito à vista dia 18 de março e hoje dia 22 /03 ainda nem postaram  o produto tento falar com a empresa casa das unhas nem respondem , preciso saber quando iram postar nos correios',
 'Fiz a compra de um esguicho hoje.Não recebi o e-mail de confirmação do pedido, mas o pagamento já foi aprovado pelo cartão, porém não recebi nada  nem número do pedido nem a  confirmação da compra. Por favor aguardo respostas',
 'Fiz uma compra de um mini drone via boleto e não t